# Transforming datasets (Laptop, CPU, GPU)

This notebook transforms the:

* laptop data scraped from Scorptec

* cpu data scraped from Passmark

* gpu data from Passmark downloaded from: https://www.kaggle.com/datasets/alanjo/gpu-benchmarks

To be compatible and usable for the oncoming analysis.


The overall goal of this is to make matching cpu and gpu components with their benchmark data straightforward, and create columns for better querying.

Information about each transformation will be under their heading.

In [30]:
import pandas as pd
import numpy as np
import os, re, json

cwd = os.getcwd()

In [31]:
pd.options.mode.chained_assignment = None

## Laptop data transformation
Create columns for the laptop brand, cpu brand, and gpu brand.

Modify the cpu and gpu columns so they match the keys of the cpu and gpu datasets

Make the display size, memory, and storage size queryable as int.

In [105]:
laptop_original = pd.read_csv(cwd + '/datasets/laptop_data/scorptec2023-04-07.csv', quotechar="'")
laptop_modify = laptop_original.copy()
laptop_modify

,NAME,PRODUCTCODE,PRICE,PROCESSOR,MEMORY,GRAPHICS,STORAGE,DISPLAY
0,Gigabyte A5 K1 Black 15.6inch Ryzen 5 RTX 3060...,A5 K1-AAU1130SB,1799,Ryzen 5-5600H,"16GB , RAM",GeForce RTX 3060 Max-P Design,512GB M.2 PCIE SSD,15.6inch FHD 144Hz
1,Razer Blade 14 Black 14inch Ryzen 9 RTX 3070 T...,RZ09-0427NEA3-R3B1,2699,Ryzen 9-6900HX,16GB DDR5 RAM,GeForce RTX 3070 Ti 8GB,1TB M.2 PCIE SSD,14inch QHD 165Hz
2,MSI Katana GF66 12UD 15.6inch Core i7 RTX 3050...,Katana GF66 12UD-069AU,1549,i7-12700H,"16GB , DDR4 RAM",GeForce RTX 3050 Ti,1TB M.2 PCIE SSD,15.6inch FHD 144Hz
3,MSI Katana GF76 12UE Black 17.3inch Core i7 RT...,Katana GF76 12UE-019AU,1899,i7-12700H,"16GB , RAM",GeForce RTX 3060 6GB,1TB M.2 PCIE SSD,17.3inch FHD 144Hz
4,MSI GF63 Thin 11SC Black 15.6inch Core i5 GTX ...,GF63 Thin 11SC-1095AU,999,i5-11400H,"8GB , RAM",GeForce GTX 1650 4GB,512GB M.2 PCIE SSD,15.6inch FHD
...,...,...,...,...,...,...,...,...
683,Lenovo ThinkPad P15v G3 15.6 inch FHD i7 16GB ...,21D80022AU,2699,Core i7-12700H,"16GB RAM DDR5 ,",Nvidia T600 4GB,512GB M.2 SSD,15.6 inch FHD IPS
684,MSI Creator M16 A12UC 16 inch i7 RTX3050 Win11...,Creator M16 A12UC-237AU,2399,Core i7-12700H,"16GB RAM ,",RTX3050 4GB,512GB SSD,16 inch QHD+ 60Hz
685,"MSI CreatorPro Z17 A12UMST 17"" Core i7 32GB 1T...",CreatorPro Z17 A12UMST-218AU,6599,Core i7-12700H,"32GB DDR5 RAM ,",RTX A5500 16GB,1TB M.2 NVMe PCIe Gen4 SSD,17inch QHD+ 165Hz
686,MSI CreatorPro X17HX A13VKS 17.3inch Core i9 6...,CreatorPro X17HX A13VKS-204AU,9599,Core i9-13980HX,"64GB DDR5 RAM ,",RTX 3500 12GB,"4TB , M.2 NVMe PCIe Gen4 SSD",17.3inch UHD 144Hz


In [106]:
# Create column for laptop brand
brandsSeries = laptop_modify.NAME.str.split(n=1, expand=True)[0]
brandsSeries.name = 'LAPTOP_BRAND'
laptop_modify = pd.concat([brandsSeries, laptop_modify], axis='columns')
laptop_modify.head()

,LAPTOP_BRAND,NAME,PRODUCTCODE,PRICE,PROCESSOR,MEMORY,GRAPHICS,STORAGE,DISPLAY
0,Gigabyte,Gigabyte A5 K1 Black 15.6inch Ryzen 5 RTX 3060...,A5 K1-AAU1130SB,1799,Ryzen 5-5600H,"16GB , RAM",GeForce RTX 3060 Max-P Design,512GB M.2 PCIE SSD,15.6inch FHD 144Hz
1,Razer,Razer Blade 14 Black 14inch Ryzen 9 RTX 3070 T...,RZ09-0427NEA3-R3B1,2699,Ryzen 9-6900HX,16GB DDR5 RAM,GeForce RTX 3070 Ti 8GB,1TB M.2 PCIE SSD,14inch QHD 165Hz
2,MSI,MSI Katana GF66 12UD 15.6inch Core i7 RTX 3050...,Katana GF66 12UD-069AU,1549,i7-12700H,"16GB , DDR4 RAM",GeForce RTX 3050 Ti,1TB M.2 PCIE SSD,15.6inch FHD 144Hz
3,MSI,MSI Katana GF76 12UE Black 17.3inch Core i7 RT...,Katana GF76 12UE-019AU,1899,i7-12700H,"16GB , RAM",GeForce RTX 3060 6GB,1TB M.2 PCIE SSD,17.3inch FHD 144Hz
4,MSI,MSI GF63 Thin 11SC Black 15.6inch Core i5 GTX ...,GF63 Thin 11SC-1095AU,999,i5-11400H,"8GB , RAM",GeForce GTX 1650 4GB,512GB M.2 PCIE SSD,15.6inch FHD


In [107]:
# Create column for CPU brand
cpu_Conditions = [
    laptop_modify.PROCESSOR.str.contains('Intel|(i\d(-|\s))', case=False),
    laptop_modify.PROCESSOR.str.contains('AMD|Ryzen|R\d', case=False),
    laptop_modify.LAPTOP_BRAND == 'Apple'
]
cpu_Values = ['Intel', 'AMD', 'Apple']
laptop_modify['CPU_BRAND'] = np.select(cpu_Conditions, cpu_Values, default='Unknown')

laptop_modify.head()

/var/folders/sl/05d_jpr94jv0p51sn01y__x00000gn/T/ipykernel_13439/1165048059.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  laptop_modify.PROCESSOR.str.contains('Intel|(i\d(-|\s))', case=False),


,LAPTOP_BRAND,NAME,PRODUCTCODE,PRICE,PROCESSOR,MEMORY,GRAPHICS,STORAGE,DISPLAY,CPU_BRAND
0,Gigabyte,Gigabyte A5 K1 Black 15.6inch Ryzen 5 RTX 3060...,A5 K1-AAU1130SB,1799,Ryzen 5-5600H,"16GB , RAM",GeForce RTX 3060 Max-P Design,512GB M.2 PCIE SSD,15.6inch FHD 144Hz,AMD
1,Razer,Razer Blade 14 Black 14inch Ryzen 9 RTX 3070 T...,RZ09-0427NEA3-R3B1,2699,Ryzen 9-6900HX,16GB DDR5 RAM,GeForce RTX 3070 Ti 8GB,1TB M.2 PCIE SSD,14inch QHD 165Hz,AMD
2,MSI,MSI Katana GF66 12UD 15.6inch Core i7 RTX 3050...,Katana GF66 12UD-069AU,1549,i7-12700H,"16GB , DDR4 RAM",GeForce RTX 3050 Ti,1TB M.2 PCIE SSD,15.6inch FHD 144Hz,Intel
3,MSI,MSI Katana GF76 12UE Black 17.3inch Core i7 RT...,Katana GF76 12UE-019AU,1899,i7-12700H,"16GB , RAM",GeForce RTX 3060 6GB,1TB M.2 PCIE SSD,17.3inch FHD 144Hz,Intel
4,MSI,MSI GF63 Thin 11SC Black 15.6inch Core i5 GTX ...,GF63 Thin 11SC-1095AU,999,i5-11400H,"8GB , RAM",GeForce GTX 1650 4GB,512GB M.2 PCIE SSD,15.6inch FHD,Intel


In [108]:
# Extract cpu model
intel_regex = r'\b(([A-Z][A-Z0-9-]*|i.*)?\d{3,}\w*)'
amd_regex = r'\b(([A-Z]+-)?\d{3,}\w*)'
apple_regex = r'\b(M\d.*Core)'

def extract_laptop_cpu(row):
    processor = row.PROCESSOR
    cpuBrand = row.CPU_BRAND

    if cpuBrand == 'Intel':
        modelMatch = re.search(intel_regex, processor)
        if modelMatch != None:
            return modelMatch.group(0)

    elif cpuBrand == 'AMD':
        modelMatch = re.search(amd_regex, processor)
        if modelMatch != None:
            return modelMatch.group(0)

    elif cpuBrand == 'Apple':
        modelMatch = re.search(apple_regex, processor)
        if modelMatch != None:
            return modelMatch.group(0)

    return processor

laptop_modify['CPU_MODEL'] = laptop_modify.apply(extract_laptop_cpu, axis = 1)
laptop_modify.head()

,LAPTOP_BRAND,NAME,PRODUCTCODE,PRICE,PROCESSOR,MEMORY,GRAPHICS,STORAGE,DISPLAY,CPU_BRAND,CPU_MODEL
0,Gigabyte,Gigabyte A5 K1 Black 15.6inch Ryzen 5 RTX 3060...,A5 K1-AAU1130SB,1799,Ryzen 5-5600H,"16GB , RAM",GeForce RTX 3060 Max-P Design,512GB M.2 PCIE SSD,15.6inch FHD 144Hz,AMD,5600H
1,Razer,Razer Blade 14 Black 14inch Ryzen 9 RTX 3070 T...,RZ09-0427NEA3-R3B1,2699,Ryzen 9-6900HX,16GB DDR5 RAM,GeForce RTX 3070 Ti 8GB,1TB M.2 PCIE SSD,14inch QHD 165Hz,AMD,6900HX
2,MSI,MSI Katana GF66 12UD 15.6inch Core i7 RTX 3050...,Katana GF66 12UD-069AU,1549,i7-12700H,"16GB , DDR4 RAM",GeForce RTX 3050 Ti,1TB M.2 PCIE SSD,15.6inch FHD 144Hz,Intel,i7-12700H
3,MSI,MSI Katana GF76 12UE Black 17.3inch Core i7 RT...,Katana GF76 12UE-019AU,1899,i7-12700H,"16GB , RAM",GeForce RTX 3060 6GB,1TB M.2 PCIE SSD,17.3inch FHD 144Hz,Intel,i7-12700H
4,MSI,MSI GF63 Thin 11SC Black 15.6inch Core i5 GTX ...,GF63 Thin 11SC-1095AU,999,i5-11400H,"8GB , RAM",GeForce GTX 1650 4GB,512GB M.2 PCIE SSD,15.6inch FHD,Intel,i5-11400H


In [109]:
# Create gpu brand column
gpu_Conditions = [
    laptop_modify.GRAPHICS.str.contains('Intel|UHD|Iris|Irix', case = False),
    laptop_modify.GRAPHICS.str.contains('Nvidia|GeForce|GTX|MX|Quadro', case = False),
    laptop_modify.GRAPHICS.str.contains('AMD|Radeon|RTX', case=False),
    laptop_modify.LAPTOP_BRAND == 'Apple'
]
gpu_Values = ['Intel', 'Nvidia', 'AMD', 'Apple']

laptop_modify['GPU_BRAND'] = np.select(gpu_Conditions, gpu_Values, default='Unknown')
laptop_modify.head()

,LAPTOP_BRAND,NAME,PRODUCTCODE,PRICE,PROCESSOR,MEMORY,GRAPHICS,STORAGE,DISPLAY,CPU_BRAND,CPU_MODEL,GPU_BRAND
0,Gigabyte,Gigabyte A5 K1 Black 15.6inch Ryzen 5 RTX 3060...,A5 K1-AAU1130SB,1799,Ryzen 5-5600H,"16GB , RAM",GeForce RTX 3060 Max-P Design,512GB M.2 PCIE SSD,15.6inch FHD 144Hz,AMD,5600H,Nvidia
1,Razer,Razer Blade 14 Black 14inch Ryzen 9 RTX 3070 T...,RZ09-0427NEA3-R3B1,2699,Ryzen 9-6900HX,16GB DDR5 RAM,GeForce RTX 3070 Ti 8GB,1TB M.2 PCIE SSD,14inch QHD 165Hz,AMD,6900HX,Nvidia
2,MSI,MSI Katana GF66 12UD 15.6inch Core i7 RTX 3050...,Katana GF66 12UD-069AU,1549,i7-12700H,"16GB , DDR4 RAM",GeForce RTX 3050 Ti,1TB M.2 PCIE SSD,15.6inch FHD 144Hz,Intel,i7-12700H,Nvidia
3,MSI,MSI Katana GF76 12UE Black 17.3inch Core i7 RT...,Katana GF76 12UE-019AU,1899,i7-12700H,"16GB , RAM",GeForce RTX 3060 6GB,1TB M.2 PCIE SSD,17.3inch FHD 144Hz,Intel,i7-12700H,Nvidia
4,MSI,MSI GF63 Thin 11SC Black 15.6inch Core i5 GTX ...,GF63 Thin 11SC-1095AU,999,i5-11400H,"8GB , RAM",GeForce GTX 1650 4GB,512GB M.2 PCIE SSD,15.6inch FHD,Intel,i5-11400H,Nvidia


In [110]:
# Load in UHD Data
# UHD_Graphics.json contains the UHD Generation each UHD Intel Chip belongs to
f = open('UHD_Graphics.json')
UHD_Data = json.load(f)
f.close()

In [111]:
# Extract gpu model
def extract_laptop_gpu(row):
    gpu_regex = r'\b[A-Z0-9-]*\d\d[A-Z0-9-]*\b'
    brand = row.GPU_BRAND
    name = row.GRAPHICS

    if brand == 'Nvidia':
        gpu_model = re.search(gpu_regex, name)
        if gpu_model != None:
            gpu_model = gpu_model.group(0)
            if 'Ti' in name:
                gpu_model += ' Ti'
            if 'Super' in name:
                gpu_model += ' Super'
            return gpu_model
        elif 'Ti' in name:
            gpuNameList = name.split('Ti')
            gpu_model = re.search(gpu_regex, gpuNameList[0]).group(0)
            gpu_model += ' Ti'
            return gpu_model
        else:
            return 'Unknown'
        
    elif brand == 'AMD':
        amd_brand_regex = 'Radeon (R\d )?Graphics'
        if re.search(amd_brand_regex, name) != None:
            return row.CPU_MODEL
        gpu_model = re.search(gpu_regex, name)
        if gpu_model != None:
            gpu_model = gpu_model.group(0)
            if 'XT' in name:
                gpu_model += ' XT'
            return gpu_model
        else:
            return 'Unknown'
        
    elif brand == 'Intel':
        gpuNameList = name.split()
        if 'Iris' in name or 'Irix' in name:
            if 'Intel' in gpuNameList:
                gpuNameList.remove('Intel')
            if 'Graphics' in gpuNameList:
                gpuNameList.remove("Graphics")
            if 'Irix' in gpuNameList:
                # Fix Irix type
                gpuNameList[gpuNameList.index('Irix')] = 'Iris'
            if 'X' in gpuNameList:
                # Fix X type
                gpuNameList[gpuNameList.index('X')] = 'Xe'
            return ' '.join(gpuNameList)
        elif 'UHD' in gpuNameList:
            # Search through  UHD json for corresponding UHD Graphic series
            for series in UHD_Data:
                if row.CPU_MODEL in UHD_Data[series]:
                    return series
            return 'UHD'
        else:
            return 'Unknown'
    
    elif brand == 'Apple':
        if 'GPU' in row.GRAPHICS:
            return row.GRAPHICS
        else:
            return row.CPU_MODEL     

    else:
        return 'Unknown'

laptop_modify['GPU_MODEL'] = laptop_modify.apply(extract_laptop_gpu, axis=1)
laptop_modify.head()

,LAPTOP_BRAND,NAME,PRODUCTCODE,PRICE,PROCESSOR,MEMORY,GRAPHICS,STORAGE,DISPLAY,CPU_BRAND,CPU_MODEL,GPU_BRAND,GPU_MODEL
0,Gigabyte,Gigabyte A5 K1 Black 15.6inch Ryzen 5 RTX 3060...,A5 K1-AAU1130SB,1799,Ryzen 5-5600H,"16GB , RAM",GeForce RTX 3060 Max-P Design,512GB M.2 PCIE SSD,15.6inch FHD 144Hz,AMD,5600H,Nvidia,3060
1,Razer,Razer Blade 14 Black 14inch Ryzen 9 RTX 3070 T...,RZ09-0427NEA3-R3B1,2699,Ryzen 9-6900HX,16GB DDR5 RAM,GeForce RTX 3070 Ti 8GB,1TB M.2 PCIE SSD,14inch QHD 165Hz,AMD,6900HX,Nvidia,3070 Ti
2,MSI,MSI Katana GF66 12UD 15.6inch Core i7 RTX 3050...,Katana GF66 12UD-069AU,1549,i7-12700H,"16GB , DDR4 RAM",GeForce RTX 3050 Ti,1TB M.2 PCIE SSD,15.6inch FHD 144Hz,Intel,i7-12700H,Nvidia,3050 Ti
3,MSI,MSI Katana GF76 12UE Black 17.3inch Core i7 RT...,Katana GF76 12UE-019AU,1899,i7-12700H,"16GB , RAM",GeForce RTX 3060 6GB,1TB M.2 PCIE SSD,17.3inch FHD 144Hz,Intel,i7-12700H,Nvidia,3060
4,MSI,MSI GF63 Thin 11SC Black 15.6inch Core i5 GTX ...,GF63 Thin 11SC-1095AU,999,i5-11400H,"8GB , RAM",GeForce GTX 1650 4GB,512GB M.2 PCIE SSD,15.6inch FHD,Intel,i5-11400H,Nvidia,1650


In [112]:
# Show laptops with incorrect/missing data
laptop_modify[laptop_modify.GPU_MODEL == 'Unknown']

,LAPTOP_BRAND,NAME,PRODUCTCODE,PRICE,PROCESSOR,MEMORY,GRAPHICS,STORAGE,DISPLAY,CPU_BRAND,CPU_MODEL,GPU_BRAND,GPU_MODEL
8,MSI,MSI Katana GF66 12UC Black 15.6inch Core i7 RT...,Katana GF66 12UC-656AU,1349,i7-12650H,"8GB , RAM",GBLAN,15.6inch FHD 144Hz,GeForce RTX 3050 4GB,Intel,i7-12650H,Unknown,Unknown
41,ASUS,ASUS ROG Zephyrus Duo 16 Black 16inch Ryzen 9 ...,GX650RX-LB149W,5599,Ryzen 9-6900HX,"32GB , DDR5 RAM",ScreenPad Plus 14inch 4k,2TB M.2 PCIE SSD,16inch UHD+ 120Hz and FHD+ 240Hz,AMD,6900HX,Unknown,Unknown
101,Gigabyte,Gigabyte G5 KD Black 15.6inch Core i5 RTX 3060...,G5 KD-52AU123SO,1799,i5-11400H,"16GB , RAM",GBLAN,15.6inch FHD 144Hz,GeForce RTX 3060 Max-P Design,Intel,i5-11400H,Unknown,Unknown
118,ASUS,ASUS ROG Zephyrus Duo 16 Black 16inch Ryzen 9 ...,GX650PY-NM056W,6999,Ryzen 9-7945HX,"64GB , DDR5 RAM",14inch ScreenPad Plus 4k,1TB M.2 PCIE SSD,16inch QHD+ 240Hz,AMD,7945HX,Unknown,Unknown
271,Asus,Asus ZenBook ProDuo 15 OLED 15.6in UHD Touch C...,UX582ZM-H2009X,4399,i9-12900HK,32GB RAM,ScreenPad Duo,1TB PCIe NVMe SSD,15.6 inch 4K UHD OLED Touch,Intel,i9-12900HK,Unknown,Unknown
283,Microsoft,Microsoft Surface Laptop 5 13.5inch Core i5 16...,R7B-00016,2399,"256GB SSD ,",13.5-inch PixelSense Gorilla Glass 3 Touch Dis...,HD Cam,Intel Iris Xe,WIFI 6 + BT 5.1,Unknown,"256GB SSD ,",Unknown,Unknown
287,ASUS,ASUS ZenBook Pro Duo 15 OLED 15.6 inch i7 16GB...,UX582ZM-KY012W,3799,Core i7-12700H,16GB RAM onboard,ScreenPad Plus,1TB PCe Gen4 SSD,15.6 inch FHD OLED Touch,Intel,i7-12700H,Unknown,Unknown
351,Asus,Asus Vivobook S 15.6 inch FHD OLED i5 8GB 256G...,K3502ZA-L1367W,1499,Neutral Grey,Core i5-12500H,15.6 inch FHD OLED,"8GB ,",256GB M.2 SSD,Unknown,Neutral Grey,Unknown,Unknown
521,Microsoft,Microsoft Surface Laptop 5 13.5inch Core i7 16...,RB1-00039,2699,"256GB SSD ,",13.5-inch PixelSense Gorilla Glass 3 Touch Dis...,HD Cam,Intel Iris Xe,WIFI 6 + BT 5.1,Unknown,"256GB SSD ,",Unknown,Unknown
522,Microsoft,Microsoft Surface Laptop 5 13.5inch Core i7 16...,RBH-00041,2849,"512GB SSD ,",13.5-inch PixelSense Gorilla Glass 5 Touch Dis...,HD Cam,Intel Iris Xe,WIFI 6 + BT 5.1,Unknown,"512GB SSD ,",Unknown,Unknown


In [114]:
# Remove Laptops with incorrect/missing data
laptop_modify.drop(laptop_modify[laptop_modify.GPU_MODEL == 'Unknown'].index, inplace=True)

In [136]:
# Clean up Memory Column and create RAM column
laptop_modify.MEMORY = laptop_modify.MEMORY.map(lambda s: s.replace(',', '').strip())
laptop_modify['RAM'] = laptop_modify.MEMORY.str.extract(r'(^\d+)')
laptop_modify.RAM = laptop_modify.RAM.astype(int)
laptop_modify.head()

,LAPTOP_BRAND,NAME,PRODUCTCODE,PRICE,PROCESSOR,MEMORY,GRAPHICS,STORAGE,DISPLAY,CPU_BRAND,CPU_MODEL,GPU_BRAND,GPU_MODEL,RAM
0,Gigabyte,Gigabyte A5 K1 Black 15.6inch Ryzen 5 RTX 3060...,A5 K1-AAU1130SB,1799,Ryzen 5-5600H,16GB RAM,GeForce RTX 3060 Max-P Design,512GB M.2 PCIE SSD,15.6inch FHD 144Hz,AMD,5600H,Nvidia,3060,16
1,Razer,Razer Blade 14 Black 14inch Ryzen 9 RTX 3070 T...,RZ09-0427NEA3-R3B1,2699,Ryzen 9-6900HX,16GB DDR5 RAM,GeForce RTX 3070 Ti 8GB,1TB M.2 PCIE SSD,14inch QHD 165Hz,AMD,6900HX,Nvidia,3070 Ti,16
2,MSI,MSI Katana GF66 12UD 15.6inch Core i7 RTX 3050...,Katana GF66 12UD-069AU,1549,i7-12700H,16GB DDR4 RAM,GeForce RTX 3050 Ti,1TB M.2 PCIE SSD,15.6inch FHD 144Hz,Intel,i7-12700H,Nvidia,3050 Ti,16
3,MSI,MSI Katana GF76 12UE Black 17.3inch Core i7 RT...,Katana GF76 12UE-019AU,1899,i7-12700H,16GB RAM,GeForce RTX 3060 6GB,1TB M.2 PCIE SSD,17.3inch FHD 144Hz,Intel,i7-12700H,Nvidia,3060,16
4,MSI,MSI GF63 Thin 11SC Black 15.6inch Core i5 GTX ...,GF63 Thin 11SC-1095AU,999,i5-11400H,8GB RAM,GeForce GTX 1650 4GB,512GB M.2 PCIE SSD,15.6inch FHD,Intel,i5-11400H,Nvidia,1650,8


In [142]:
# Clean up Storage column and create STORAGE_SIZE column
laptop_modify.STORAGE = laptop_modify.STORAGE.map(lambda s: s.replace(',', '').strip())

def calc_storage(row):
    totalStorage = 0
    hd_regex = r'(\d+)\s?(GB|TB)'
    storage = row.STORAGE
    storageList = storage.split('+')
    storageList = [item.strip() for item in storageList]
    for hardDrive in storageList:
        hdStorage = re.search(hd_regex, hardDrive)
        if 'TB' in hdStorage.group(0):
            hdStorage = int(hdStorage.group(1))
            hdStorage = hdStorage*1000
        else:
            hdStorage = int(hdStorage.group(1))
        totalStorage += int(hdStorage)

    return totalStorage

laptop_modify['STORAGE_SIZE'] = laptop_modify.apply(calc_storage, axis=1)
laptop_modify

,LAPTOP_BRAND,NAME,PRODUCTCODE,PRICE,PROCESSOR,MEMORY,GRAPHICS,STORAGE,DISPLAY,CPU_BRAND,CPU_MODEL,GPU_BRAND,GPU_MODEL,RAM,STORAGE_SIZE
0,Gigabyte,Gigabyte A5 K1 Black 15.6inch Ryzen 5 RTX 3060...,A5 K1-AAU1130SB,1799,Ryzen 5-5600H,16GB RAM,GeForce RTX 3060 Max-P Design,512GB M.2 PCIE SSD,15.6inch FHD 144Hz,AMD,5600H,Nvidia,3060,16,512
1,Razer,Razer Blade 14 Black 14inch Ryzen 9 RTX 3070 T...,RZ09-0427NEA3-R3B1,2699,Ryzen 9-6900HX,16GB DDR5 RAM,GeForce RTX 3070 Ti 8GB,1TB M.2 PCIE SSD,14inch QHD 165Hz,AMD,6900HX,Nvidia,3070 Ti,16,1000
2,MSI,MSI Katana GF66 12UD 15.6inch Core i7 RTX 3050...,Katana GF66 12UD-069AU,1549,i7-12700H,16GB DDR4 RAM,GeForce RTX 3050 Ti,1TB M.2 PCIE SSD,15.6inch FHD 144Hz,Intel,i7-12700H,Nvidia,3050 Ti,16,1000
3,MSI,MSI Katana GF76 12UE Black 17.3inch Core i7 RT...,Katana GF76 12UE-019AU,1899,i7-12700H,16GB RAM,GeForce RTX 3060 6GB,1TB M.2 PCIE SSD,17.3inch FHD 144Hz,Intel,i7-12700H,Nvidia,3060,16,1000
4,MSI,MSI GF63 Thin 11SC Black 15.6inch Core i5 GTX ...,GF63 Thin 11SC-1095AU,999,i5-11400H,8GB RAM,GeForce GTX 1650 4GB,512GB M.2 PCIE SSD,15.6inch FHD,Intel,i5-11400H,Nvidia,1650,8,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,Lenovo,Lenovo ThinkPad P15v G3 15.6 inch FHD i7 16GB ...,21D80022AU,2699,Core i7-12700H,16GB RAM DDR5,Nvidia T600 4GB,512GB M.2 SSD,15.6 inch FHD IPS,Intel,i7-12700H,Nvidia,T600,16,512
684,MSI,MSI Creator M16 A12UC 16 inch i7 RTX3050 Win11...,Creator M16 A12UC-237AU,2399,Core i7-12700H,16GB RAM,RTX3050 4GB,512GB SSD,16 inch QHD+ 60Hz,Intel,i7-12700H,AMD,RTX3050,16,512
685,MSI,"MSI CreatorPro Z17 A12UMST 17"" Core i7 32GB 1T...",CreatorPro Z17 A12UMST-218AU,6599,Core i7-12700H,32GB DDR5 RAM,RTX A5500 16GB,1TB M.2 NVMe PCIe Gen4 SSD,17inch QHD+ 165Hz,Intel,i7-12700H,AMD,A5500,32,1000
686,MSI,MSI CreatorPro X17HX A13VKS 17.3inch Core i9 6...,CreatorPro X17HX A13VKS-204AU,9599,Core i9-13980HX,64GB DDR5 RAM,RTX 3500 12GB,4TB M.2 NVMe PCIe Gen4 SSD,17.3inch UHD 144Hz,Intel,i9-13980HX,AMD,3500,64,4000


In [143]:
print(laptop_modify.DISPLAY.to_string())

0                                     15.6inch FHD 144Hz
1                                       14inch QHD 165Hz
2                                     15.6inch FHD 144Hz
3                                     17.3inch FHD 144Hz
4                                           15.6inch FHD
5                                     15.6inch FHD 144Hz
6                                     15.6inch FHD 144Hz
7                                     17.3inch FHD 144Hz
9                                     17.3inch FHD 144Hz
10                                    17.3inch FHD 144Hz
11                                    15.6inch FHD 144Hz
12                                    17.3inch QHD 240Hz
13                                    15.6inch FHD 240Hz
14                                   15.6inch WQHD 165Hz
15                                  15.6" OLED QHD 240Hz
16                                    17.3inch FHD 144Hz
17                                    15.6inch FHD 144Hz
18                             

In [ ]:
# Create DISPLAY_SIZE column
laptop_modify['DISPLAY_SIZE'] = laptop_modify.DISPLAY

## CPU data transformation

Add brand column to cpu data.

Extract processor model as primary key.

In [45]:
cpu_original = pd.read_csv(cwd + '/datasets/processor_data/cpu_data2023-03-21.csv', quotechar="'")
cpu_modify = cpu_original.copy()
# Convert CPU Mark to integer
cpu_modify['CPU Mark'] = cpu_modify['CPU Mark'].str.replace(',', '').astype(int)
cpu_modify

,CPU Name,CPU Mark
0,AArch64 rev 2 (aarch64),2267
1,AArch64 rev 4 (aarch64),1824
2,AC8257V/WAB,711
3,AMD 3015Ce,2067
4,AMD 3015e,2709
...,...,...
4104,ZHAOXIN KaiXian KX-6640MA@2.2+GHz,1549
4105,ZHAOXIN KaiXian KX-U6580@2.5GHz,3227
4106,ZHAOXIN KaiXian KX-U6780A@2.7GHz,3881
4107,ZHAOXIN KaiXian ZX-C+ C4700@2.0GHz,1547


Remove CPU models not offered for current laptops

*Note: Filtering based of anecdotal observations. A better way of filtering would be to include the 'CPU First Seen on Charts' and 'CPU Class' in the scraped data, and filter based off these two columns. However, the anecdotal filter is sufficient for the current dataset and the scope of the project currently.*

In [46]:
# Remove processors not from: Intel, AMD, Apple
cpu_modify = cpu_modify.drop(cpu_modify[~cpu_modify['CPU Name'].str.contains('Intel|AMD|Apple')].index)
# Remove processors that are not within the following lineup-
# Intel: Celeron, Pentium, Core, Xeon
# AMD: Ryzen, Athlon
# Apple: Apple silicon (M series)
cpu_modify = cpu_modify.drop(cpu_modify[~cpu_modify['CPU Name'].str.contains('Celeron|Pentium|Core|Xeon|Ryzen|Athlon|Apple| M\d')].index)
# Remove remaining mobile processors
cpu_modify = cpu_modify.drop(cpu_modify[cpu_modify['CPU Name'].str.contains('Mobile')].index)
# Remove CPUs with a CPU Mark below 1500
cpu_modify = cpu_modify.drop(cpu_modify[cpu_modify['CPU Mark']<1900].index)
cpu_modify

,CPU Name,CPU Mark
169,AMD Athlon 200GE,4102
170,AMD Athlon 220GE,4424
171,AMD Athlon 240GE,4534
175,AMD Athlon 3000G,4482
176,AMD Athlon 300GE,4256
...,...,...
3599,Intel Xeon X5680 @ 3.33GHz,6794
3600,Intel Xeon X5687 @ 3.60GHz,5288
3601,Intel Xeon X5690 @ 3.47GHz,6967
3602,Intel Xeon X5698 @ 4.40GHz,3447


In [47]:
# Remove the unnecessary clockspeed in some of the CPU Names
cpu_modify['CPU Name'] = cpu_modify['CPU Name'].str.split(pat='@', expand=True)[0]
cpu_modify['CPU Name'].str.strip()

# Create Column for CPU Brand
cpu_Conditions = [
    cpu_modify['CPU Name'].str.contains('Intel', case=False),
    cpu_modify['CPU Name'].str.contains('AMD', case=False),
    cpu_modify['CPU Name'].str.contains('Apple', case=False)
]
cpu_Values = ['Intel', 'AMD', 'Apple']

cpu_modify['CPU Brand'] = np.select(cpu_Conditions, cpu_Values, default='Unknown')
cpu_modify

,CPU Name,CPU Mark,CPU Brand
169,AMD Athlon 200GE,4102,AMD
170,AMD Athlon 220GE,4424,AMD
171,AMD Athlon 240GE,4534,AMD
175,AMD Athlon 3000G,4482,AMD
176,AMD Athlon 300GE,4256,AMD
...,...,...,...
3599,Intel Xeon X5680,6794,Intel
3600,Intel Xeon X5687,5288,Intel
3601,Intel Xeon X5690,6967,Intel
3602,Intel Xeon X5698,3447,Intel


In [48]:
# Create column for CPU Model and set as index
intel_regex = r'\s(([A-Z][A-Z0-9-]*|i.*)?\d{3,}\w*)'
amd_regex = r'\s((\w+-)?\d{3,}\w*)'
apple_regex = r'\s(M\d.*Core)'

def extract_cpu_model(row):
    cpu_name = row['CPU Name']
    cpu_brand = row['CPU Brand']

    if cpu_brand == 'Intel':
        match = re.search(intel_regex, cpu_name)
        if match:
            return match.group(1)
    if cpu_brand == 'AMD':
        match = re.search(amd_regex, cpu_name)
        if match:
            return match.group(1)
    if cpu_brand == 'Apple':
        match = re.search(apple_regex, cpu_name)
        if match:
            return match.group(1)
        
    return 'Unknown'

cpu_modify['CPU Model'] = cpu_modify.apply(extract_cpu_model, axis = 1)

# Entries where the CPU Model extraction failed
cpu_modify = cpu_modify.drop(cpu_modify[cpu_modify['CPU Model'] == 'Unknown'].index)

# Duplicate check
cpu_modify['CPU Model'].value_counts()

E3-1225     5
E3-1220     5
E3-1230     5
E3-1275     5
E3-1240     5
           ..
i5-3320M    1
i5-3317U    1
i5-3230M    1
i5-3210M    1
X6550       1
Name: CPU Model, Length: 1655, dtype: int64

In [49]:
def agg_cpu_model(group):
    if len(group) == 1:
        return group.iloc[0]
    else:
        # For duplicate values, it's presumably the same CPU under two different CPU names.
        # Take the shorter of the two names and average the CPU Mark.
        # Note: Averaging the CPU Marks is sufficient for practical purposes as the two values should be similar.
        # If we wanted to be pedantic, PassMark shows the number of samples for each CPU Mark and we could recalculate a precise value.
        shorter_name_idx = group['CPU Name'].str.len().idxmin()
        shorter_name = group.loc[shorter_name_idx, 'CPU Name']
        avg_mark = group['CPU Mark'].mean()
        brand = group.iloc[0]['CPU Brand']
        return pd.Series({'CPU Name': shorter_name, 'CPU Mark': avg_mark, 'CPU Brand': brand})

cpu_modify = cpu_modify.groupby('CPU Model').apply(agg_cpu_model)
cpu_modify = cpu_modify.unstack()
cpu_modify.drop(columns='CPU Model', inplace=True)
cpu_modify

,CPU Brand,CPU Mark,CPU Name
CPU Model,,,
1200,AMD,6309.5,AMD Ryzen 3 1200
1300,AMD,7234,AMD Ryzen 3 PRO 1300
1300X,AMD,6957,AMD Ryzen 3 1300X
1400,AMD,7756,AMD Ryzen 5 1400
1403,Intel,2284,Intel Pentium 1403 v2
...,...,...,...
i9-9940X,Intel,28402,Intel Core i9-9940X
i9-9960X,Intel,30354,Intel Core i9-9960X
i9-9980HK,Intel,14493,Intel Core i9-9980HK


# GPU data transformation
Add brand column to the dataset.

Extract GPU Model from GPU Name.

In [120]:
gpu_original = pd.read_csv(cwd + '/datasets/downloaded_data/GPU_Benchmarks_Compilation/GPU_benchmarks_v7.csv', quotechar='"')
gpu_modify = gpu_original.copy()
gpu_modify

,gpuName,G3Dmark,G2Dmark,price,gpuValue,TDP,powerPerformance,testDate,category
0,GeForce RTX 3090 Ti,29094,1117,2099.99,13.85,450.0,64.65,2022,Unknown
1,GeForce RTX 3080 Ti,26887,1031,1199.99,22.41,350.0,76.82,2021,Desktop
2,GeForce RTX 3090,26395,999,1749.99,15.08,350.0,75.41,2020,Desktop
3,Radeon RX 6900 XT,25458,1102,1120.31,22.72,300.0,84.86,2020,Desktop
4,GeForce RTX 3080,24853,1003,999.00,24.88,320.0,77.66,2020,Desktop
...,...,...,...,...,...,...,...,...,...
2312,Intel 82852/82855 GM/GME Controller,1,107,NaN,NaN,NaN,NaN,2010,Unknown
2313,Quadro2 Pro,1,143,NaN,NaN,NaN,NaN,2009,Workstation
2314,Rage 128 Pro,1,40,NaN,NaN,NaN,NaN,2009,Unknown
2315,RAGE 128 PRO AGP 4X TMDS,1,142,NaN,NaN,NaN,NaN,2009,Unknown


In [121]:
# Remove irrelevant rows and columns
gpu_modify = gpu_modify[(gpu_modify.category != 'Desktop') & (gpu_modify.category != 'Workstation')]
gpu_modify.drop(gpu_modify[gpu_modify['gpuName'].str.contains('\+|/')].index, inplace=True)
gpu_modify.drop(gpu_modify[~gpu_modify['gpuName'].str.contains('Intel|AMD|Nvidia|GeForce|Radeon|RTX|GTX', case=False)].index, inplace=True)
gpu_modify.drop(gpu_modify[gpu_modify['gpuName'].str.contains('Radeon HD', case=False)].index, inplace=True)
gpu_modify.drop(columns=['G2Dmark', 'price', 'gpuValue', 'TDP', 'powerPerformance', 'testDate'], inplace=True)
gpu_modify.drop(gpu_modify[gpu_modify['G3Dmark']<700].index, inplace=True)
gpu_modify.head()

,gpuName,G3Dmark,category
0,GeForce RTX 3090 Ti,29094,Unknown
11,RTX A4500,21546,Unknown
18,GeForce RTX 3080 Ti Laptop GPU,19507,Unknown
23,Radeon PRO W6800,18802,Unknown
26,GeForce RTX 3070 Ti Laptop GPU,18490,Unknown


In [122]:
# Create brand column
gpu_Conditions = [
    gpu_modify.gpuName.str.contains('Intel', case = False),
    gpu_modify.gpuName.str.contains('Nvidia|GeForce|GTX', case = False),
    gpu_modify.gpuName.str.contains('AMD|Radeon|RTX', case=False)
]
gpu_Values = ['Intel', 'Nvidia', 'AMD']

gpu_modify['gpuBrand'] = np.select(gpu_Conditions, gpu_Values, default='Unknown')
gpu_modify.head(0)

,gpuName,G3Dmark,category,gpuBrand


In [123]:
# Extract gpu model
def extract_gpu_model(row):
    gpu_regex = r'\b[A-Z0-9-]*\d\d[A-Z0-9-]*\b'
    brand = row.gpuBrand
    name = row.gpuName

    if brand == 'Nvidia':
        gpu_model = re.search(gpu_regex, name)
        if gpu_model != None:
            gpu_model = gpu_model.group(0)
            if 'Ti' in name:
                gpu_model += ' Ti'
            if 'Super' in name:
                gpu_model += ' Super'
            return gpu_model
        else:
            return 'Unknown model'
        
    elif brand == 'AMD':
        gpu_model = re.search(gpu_regex, name)
        if gpu_model != None:
            gpu_model = gpu_model.group(0)
            if 'XT' in name:
                gpu_model += ' XT'
            return gpu_model
        else:
            return 'Unknown model'
        
    elif brand == 'Intel':
        intelName = name.split(' ')
        gpu_model = ' '.join(intelName[1:])
        return gpu_model

    else:
        return 'Unknown brand'

gpu_modify['gpuModel'] = gpu_modify.apply(extract_gpu_model, axis=1)
gpu_modify.head()

,gpuName,G3Dmark,category,gpuBrand,gpuModel
0,GeForce RTX 3090 Ti,29094,Unknown,Nvidia,3090 Ti
11,RTX A4500,21546,Unknown,AMD,A4500
18,GeForce RTX 3080 Ti Laptop GPU,19507,Unknown,Nvidia,3080 Ti
23,Radeon PRO W6800,18802,Unknown,AMD,W6800
26,GeForce RTX 3070 Ti Laptop GPU,18490,Unknown,Nvidia,3070 Ti


In [124]:
# Check for duplicate gpu models
duplicateGpu = gpu_modify.groupby(by=['gpuBrand', 'gpuModel']).filter(lambda x: len(x)>1)
duplicateGpu = duplicateGpu[duplicateGpu.gpuModel != 'Unknown model']
print(duplicateGpu.to_string())

                                      gpuName  G3Dmark             category gpuBrand   gpuModel
52                  GeForce RTX 2080 (Mobile)    15107               Mobile   Nvidia       2080
54                   Quadro RTX 5000 (Mobile)    14832  Mobile, Workstation      AMD       5000
65          Quadro RTX 5000 with Max-Q Design    13893  Mobile, Workstation      AMD       5000
70                             RTX A2000 12GB    13688              Unknown      AMD      A2000
71         GeForce RTX 2080 with Max-Q Design    13681               Mobile   Nvidia       2080
74          Quadro RTX 4000 with Max-Q Design    13622               Mobile      AMD       4000
77                       RTX A3000 Laptop GPU    13263  Mobile, Workstation      AMD      A3000
80                                  RTX A2000    13128              Unknown      AMD      A2000
81                   Quadro RTX 4000 (Mobile)    13008               Mobile      AMD       4000
84                           GeForce RTX

In [125]:
# Set the index to gpuName for dealing with duplicates
gpu_modify = gpu_modify.set_index('gpuName')

In [126]:
# Clean up duplicate gpu models
# If there are specified laptop versions, use the laptop versions
# Else use the non-laptop versions
# Average the marks across the duplicates
for model in duplicateGpu.gpuModel.unique():
    duplicateNames = gpu_modify[gpu_modify.gpuModel == model].index.values

    laptopInName = []
    laptopNotInName = []
    for gpuName in duplicateNames:
        if 'Laptop' in gpuName:
            laptopInName += [gpuName]
        else:
            laptopNotInName += [gpuName]

    if laptopInName != []:
        # remove laptop not in name
        averageMark = gpu_modify.loc[laptopInName].G3Dmark.mean()
        gpu_modify.drop(laptopNotInName, inplace=True)
        selectName = {'name': '', 'length': 999}
        for name in laptopInName:
            if (len(name)) < selectName['length']:
                selectName['length'] = len(name)
                selectName['name'] = name
        laptopInName.remove(selectName['name'])
        gpu_modify.at[selectName['name'], 'G3Dmark'] = averageMark
        gpu_modify.drop(laptopInName, inplace=True)
    else:
        averageMark = gpu_modify.loc[laptopNotInName].G3Dmark.mean()
        selectName = {'name': '', 'length': 999}
        for name in laptopNotInName:
            if (len(name)) < selectName['length']:
                selectName['length'] = len(name)
                selectName['name'] = name
        laptopNotInName.remove(selectName['name'])
        gpu_modify.at[selectName['name'], 'G3Dmark'] = averageMark
        gpu_modify.drop(laptopNotInName, inplace=True)

gpu_modify

,G3Dmark,category,gpuBrand,gpuModel
gpuName,,,,
GeForce RTX 3090 Ti,29094.0,Unknown,Nvidia,3090 Ti
RTX A4500,21546.0,Unknown,AMD,A4500
GeForce RTX 3080 Ti Laptop GPU,19507.0,Unknown,Nvidia,3080 Ti
Radeon PRO W6800,18802.0,Unknown,AMD,W6800
GeForce RTX 3070 Ti Laptop GPU,18490.0,Unknown,Nvidia,3070 Ti
...,...,...,...,...
"Radeon R5 A10-9600P RADEON R5, 10 COMPUTE CORES 4C",716.0,Unknown,AMD,A10-9600P
Intel HD 5600,712.0,Unknown,Intel,HD 5600
Radeon R7 A8-7650K,704.0,Unknown,AMD,A8-7650K


In [128]:
print(gpu_modify.to_string())

                                                         G3Dmark             category gpuBrand                 gpuModel
gpuName                                                                                                                
GeForce RTX 3090 Ti                                 29094.000000              Unknown   Nvidia                  3090 Ti
RTX A4500                                           21546.000000              Unknown      AMD                    A4500
GeForce RTX 3080 Ti Laptop GPU                      19507.000000              Unknown   Nvidia                  3080 Ti
Radeon PRO W6800                                    18802.000000              Unknown      AMD                    W6800
GeForce RTX 3070 Ti Laptop GPU                      18490.000000              Unknown   Nvidia                  3070 Ti
Radeon Pro VII                                      16853.000000              Unknown      AMD            Unknown model
GeForce RTX 3080 Laptop GPU             